## Example interactive dqdv session

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cellpy import (cellreader, prms, log)

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
import bokeh
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
my_data = cellreader.CellpyData()
my_data.load(filename)

In [6]:
my_data.name

'20160805_test001_45_cc'

In [7]:
from cellpy.utils import ica

In [8]:
from bokeh.plotting import figure, show
import bokeh.palettes

In [9]:
# ica.dqdv??

In [10]:
# my_data.get_cap?

In [11]:
cycle_df = my_data.get_cap(1, categorical_column=True, method = "forth")
plot = figure(plot_width=800, plot_height=300)
plot.line(x=cycle_df.capacity, y=cycle_df.voltage)
show(plot)

In [12]:
def color_list(list_of_cycles):
    colors = bokeh.palettes.cividis(len(list_of_cycles))
    return colors
        


In [13]:
list_of_cycles = [1, 2, 3, 4, 5, 6, 7]
colors = color_list(list_of_cycles)

In [14]:
def _get_ica(data, c):
    cycle_df = data.get_cap(c, categorical_column=True, method = "forth-and-forth")
    return ica.dqdv_cycle(cycle_df)

In [15]:
def _to_cycles_frame(data, cycles, **kwargs):
    pass

In [16]:
def to_frame(data, cycles=None, frame_type="ica", **kwargs):
    """function that creates a dataframe of cellpy-cycle-data.
    
    Hint: For making a nice frame for plotting using Bokeh,
    its might be benificial to set long_format=True. At least
    if you plan to use HoloViews. But for exporting to Origin,
    its better to have a wide format (long_format=False).
    
    Hint: The long_format is of the form xy-xy-xy with a multi-
    index with cycle number as top-level and x and y as second
    levels. For example for ica-data you have:
    
      cycle number: | 1      | 2      | ...    |
      type:         | v | dq | v | dq | v | dq |
                    ----------------------------
                  1 | . | .  |.  | .  |.  | .  |
                  2 | . | .  |.  | .  |.  | .  |
                  .
                  n | . | .  |.  | .  |.  | .  |
    """
    
    selector = dict()
    selector["ica"] = _to_ica_frame
    selector["cycles"] = _to_cycles_frame
    
    frame_type = frame_type.lower()
    frame_engine = selector[frame_type]
    
    if cycles is None:
        cycles = data.get_cycle_numbers()
    frame = frame_engine(data, cycles, **kwargs)
    
    return frame

In [17]:
def _to_ica_frame(data, cycles, **kwargs):
    try:
        long_format = kwargs["long_format"]
    except KeyError:
        long_format = False
        
    frames = []
    if not long_format:
        keys = []

    for cycle in cycles:
        try:
            v, dq = _get_ica(data, cycle)
        except AttributeError:
            print(f"AttributeError(_to_ica_frame): missing (half-) cycle data for cycle {cycle}?")
        else:
            _df = pd.DataFrame(
                {
                    "voltage": v,
                    "dq": dq,
                },
            )

            if long_format:
                _df["cycle"] = cycle
            else:
                label = cycle
                _df.name = label
                keys.append(label)
            frames.append(_df)
    if long_format:
        df = pd.concat(frames, axis=0)
        return df
    df = pd.concat(frames,keys=keys,axis=1)
    return df

In [18]:
f_wide = to_frame(my_data, long_format=False)
f_long = to_frame(my_data, long_format=True)

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


AttributeError(_to_ica_frame): missing (half-) cycle data for cycle 18?
AttributeError(_to_ica_frame): missing (half-) cycle data for cycle 18?


In [19]:
def simple_plot(x, y):
    p = figure()
    p.line(x=x, y=y)
    show(p)

In [20]:
def simple_xyxy_plot(df):
    p = figure()
    cycles = np.unique(df.columns.get_level_values(0))
    for c in cycles:
        xy = df.loc[:, (c,slice(None))]
        p.line(x=xy.iloc[:, 0], y=xy.iloc[:, 1])
    show(p)
    

In [21]:
simple_xyxy_plot(f_wide)

In [22]:
simple_plot(f_long.voltage, f_long.dq)

In [23]:
def ica_plot(data, cycles=None, colors=None, x_scale=None, y_scale=None):
    
    if cycles is None:
        cycles = data.get_cycle_numbers()
    if colors is None:
        colors = bokeh.palettes.cividis(len(cycles))
        
    p = figure(
        plot_width=800, plot_height=600,  
        title=f"ICA for {data.name}",
        x_range = x_scale,
        y_range = y_scale,
    )

    for n, c in enumerate(cycles):
        color = colors[n]
        name = f"cycle {c}"
        v, dq = _get_ica(data, c)
        p.line(x=v, y=dq, line_color=color, legend=name)
    
    p.xaxis.axis_label = "voltage"
    p.yaxis.axis_label = "dqdv"
    return p


In [24]:
def add_legend(p):
    p.legend.location = "top_right"
    p.legend.click_policy="hide"
    return p

In [25]:
def add_ica(data, p, cycle, label=None, plot_style='scatter', **plotargs):
    if label is None:
        label = f"cycle {cycle}"
    v, dq = _get_ica(data, cycle)
    if plot_style == 'scatter':
        p.scatter(x=v, y=dq, legend=label, **plotargs)
    elif plot_style == "line":
        p.line(x=v, y=dq, legend=label, **plotargs)
    else:
        pass
    return p

In [26]:
cycles = range(1,18)

p = ica_plot(my_data, cycles, x_scale=(0,1))
p = add_ica(my_data, p, 3, label="navy", color="navy", size=2, alpha=0.3)
p = add_ica(my_data, p, 7, plot_style='line', label="new seven", line_width=4)
p = add_legend(p)
show(p)

/Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [28]:
show(p)

In [29]:
my_data.name

'20160805_test001_45_cc'